# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

So this is a estimation problem. 
The company need to estimate which preservative is responsible of the problem. We wont predict future results.
We need no analyze the dataset and find the relevant features correlated to spoiled food.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dogfood').getOrCreate()

21/11/12 12:29:25 WARN Utils: Your hostname, xtian-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.39 instead (on interface enp5s0)
21/11/12 12:29:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/12 12:29:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/12 12:29:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/12 12:29:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# Load training data
data = spark.read.csv('dog_food.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [31]:
data.count()

490

In [4]:
data.head(10)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0),
 Row(A=6, B=2, C=13.0, D=6, Spoiled=1.0),
 Row(A=4, B=2, C=12.0, D=1, Spoiled=1.0),
 Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=10, B=3, C=13.0, D=9, Spoiled=1.0),
 Row(A=8, B=5, C=14.0, D=5, Spoiled=1.0),
 Row(A=5, B=8, C=12.0, D=8, Spoiled=1.0),
 Row(A=6, B=5, C=12.0, D=9, Spoiled=1.0),
 Row(A=3, B=3, C=12.0, D=1, Spoiled=1.0)]

In [17]:
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [11]:
# Asemble features into a vector
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")

In [23]:
assembled_data = assembler.transform(data)

In [25]:
dtc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')

In [26]:
assembled_data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
# Final data must have features and spoiled.
# That's expecting the model.
final_data = assembled_data.select('features','Spoiled')
final_data.head()

Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [30]:
dtc_model = dtc.fit(final_data)

In [58]:
dtc_model.featureImportances

SparseVector(4, {1: 0.0019, 2: 0.9832, 3: 0.0149})

Given a tree ensemble model, RandomForest.featureImportances computes the importance of each feature.


In [59]:
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [60]:
rfc_model = rfc.fit(final_data)

In [66]:
feats=rfc_model.featureImportances

In [67]:
for i,feat in enumerate(feats):
    print(i, feat)
    

0 0.017869550893498946
1 0.01686581213990415
2 0.945601744447764
3 0.019662892518832888


Chemical index 2 is the cause.